## 네이버 이미지 크롤링
- selenium을 이용해 이미 크롤링
- 폴더 생성/기본폴더, 파일명에 따른 다른 이름 저장

코랩에서 selenium 사용을 위한 사전 설치

In [15]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [ ]:
# 필요한 모듈(라이브러리 연렬)
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib import request    # 이미지 다운로드에 사용
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import os

In [ ]:
# Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)

#### 준비
1. 크롬 드라이브 실행
2. 검색어를 이용한 url 생성
3. 화면 스크롤 이동을 통한 추가 이미지 생성

In [ ]:
# 키워드 입력 및 url 생성
keyword=input("이미지 검색어 입력: ")

url="https://search.naver.com/search.naver?where=image&sm=tab_jum&query="
url += keyword  # url = url + keyword

# url을 이용한 페이지 이동
driver.get(url)

#스크로 이동 횟수 정의
count_down=int(input("스크롤 다운 횟수 입력(최대:6):"))

# 사용자가 요구한 스크롤 다운 횟수 확인(최대 6회)
if count_down > 6:
    count_down = 6


# windows.scrollTo(0, Height) : 스크롤을 아래쪽으로 이동
# Height 대신 document.body.scrollHeight를 입력하면 페이지 끝으로 이동

for i in tqdm(range(count_down)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1)

이미지 검색어 입력: 선박


InvalidSessionIdException: ignored

#### 화면에서 이미지 추출하기
1. 드라이브의 소스코드 받아오기
2. 이미지가 있는 태그 찾기
3. 태그에서 이미지 소스("src") 받아오기
4. src 주소를 이용해 이미지 다운로드 하기

In [ ]:
# 드라이브 소스코드 가져오기
html = driver.page_source
soup = bs(html, "html.parser")

#### Image lazy loading
- 현재 화면에 필요한 이미지만 불러오는 기능
- 구글에서 검색해 보세요

In [ ]:
# 이미지 대표 태그 찾아 tag_img 변수에 저장(tag_img => 리스트형 데이터)
tag_img = soup.find_all("div", class_="thumb")

img=tag_img[4].find("img", class_="_image _listImage")

if img.get("data-lazy-src") == None:
    print(img["src"])
else:
    print(img.get("data-lazy-src"))

https://search.pstatic.net/sunny/?src=https%3A%2F%2Fcdn.011st.com%2F11dims%2Fresize%2F600x600%2Fquality%2F75%2F11src%2Fpd%2F22%2F5%2F9%2F8%2F1%2F7%2F3%2FxvAGw%2F233598173_B.jpg&type=a340


In [ ]:
# 태그에서 이미지 소스("src") 받아오기
tag_src=[]
for tag in tqdm(tag_img):
    img=tag.find("img", class_="_image _listImage")
    if img.get("data-lazy-src") == None:
        tag_src.append(tag.find("img")['src'])
    else:
        tag_src.append(img.get("data-lazy-src"))
        

#tag_src=[tag.find("img")['src'] for tag in tag_img]
#print(len(tag_src))
driver.close()

100%|██████████| 346/346 [00:00<00:00, 19685.42it/s]


In [ ]:
# 이미지 src를 이용해 이미지 데이터 다운로드 후 저장하기
img_name = "./pydata/" + keyword + ".jpg"

from urllib import request

# 웹상의 이미지를 다운로드후 저장
# urllib.request.urlretrieve(대상 src_url, "저장경로와 파일명") 
request.urlretrieve(tag_src[4], img_name) 

('./pydata/배.jpg', <http.client.HTTPMessage at 0x7f3b451f5bd0>)

#### 검색어를 이용해 폴더 생성후 전체 이미지 저장
- os.makedirs("폴더 경로/생성 폴더명") => 폴더 생성 명령어
- 폴더가 없으면 생성후 저장
- 기존에 폴더가 존재하며 기존 폴더이름 뒤에 1, 2, .. 와 같이 번호를 붙여 생성 


In [ ]:
# 폴더 생성 실습

os.makedirs("./pydata/imgs")
os.listdir('./pydata/')

['배.jpg', 'imgs']

In [ ]:
os.path.exists("./pydata/")   # 폴더 존재여부 확인: 있으면 True 반환, 없으면 False 반환

True

In [ ]:
# 이미지를 저장할 대표 폴더 생성(images)
fdir="./pydata/images"

if os.path.exists(fdir):  # 폴더가 있다면 뒤쪽에 "/"만 연결
    fdir += "/"
else:
    os.makedirs(fdir)     # 폴더가 없다면 폴더 생성후 뒤쪽에 "/"만 연결
    fdir += "/"

# images 폴더에 검색 키워드를 이용해 폴더 생성후 저장
# 키워드와 동일한 폴더가 있는 경우 뒤에 1씩 번호를 증가시기면서 폴더명 확인(없으면 생성)

if not os.path.exists(fdir + keyword):
    os.makedirs(fdir + keyword)     # 폴더가 없다면 폴더 생성후 뒤쪽에 "/"만 연결
    fdir = fdir + keyword +  "/"
else:
    # 폴더가 있다면 새로운 폴더 생성(번호 증가)
    num = 0
    while True:
        num += 1 #번호 1씩 증가
        
        #증가된 번호와 기존 폴더명을 연경해서 존재여부 확인
        if not os.path.exists(fdir + keyword + str(num)):
            # 없으면 폴더 생성후 while 종료
            os.makedirs(fdir + keyword + str(num)) 
            fdir = fdir + keyword + str(num) + "/"
            break        

이미지 저장
- 저장 이미지 이름 뒤에 번호 붙이기

In [ ]:
num = 1
for tag in tqdm(tag_src):
    img_name = fdir + keyword + str(num) + ".jpg"
    request.urlretrieve(tag, img_name)
    #time.sleep(1)
    num += 1

 70%|██████▉   | 241/346 [01:08<00:29,  3.52it/s]


KeyboardInterrupt: ignored

In [ ]:
tag_src[4]

'https://search.pstatic.net/common/?src=http%3A%2F%2Fimage.nmv.naver.net%2Fblog_2022_05_01_2133%2F7349dd29-c92b-11ec-bdfe-48df37ae3dc4_01.jpg&type=a340'

### [미션] 구글 이미지 크롤링
- [기본] 스크롤바를 이용해 검색한 기본 이미지를 검색키워드 폴더 생성후 저장
- [추가1] 스크롤바 이동후 [더보기] 버튼이 나오면 클릭후 나타난 모든 이미지 저장
- [추가2] PIL 모듈을 이용해 이미지 크기를 250* 250 으로 변경해 저장하시오